In [ ]:
import re

import matplotlib.pyplot as plt
from commonroad.common.util import Interval
from commonroad.scenario.scenario import Tag
from commonroad.visualization.mp_renderer import MPRenderer

from OpenSCENARIO2CR.ConversionAnalyzer.EAnalyzer import EAnalyzer
from OpenSCENARIO2CR.ConversionAnalyzer.SpotAnalyzer import SpotAnalyzerResult, SpotAnalyzer
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.EsminiWrapperProvider import EsminiWrapperProvider
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter
from OpenSCENARIO2CR.Osc2CrConverterResult import Osc2CrConverterResult
from OpenSCENARIO2CR.util.AbsRel import AbsRel
from OpenSCENARIO2CR.util.PlanningProblemBuilder import PPSBuilder

In [ ]:
scenario_path = ""
run_viewer = False
plots_step = 5
plot_limit = 20  # If non-null the renderer follows the ego vehicle
following_obstacle_index = 0

In [ ]:
# Setup EsminiWrapper
esmini_wrapper = EsminiWrapperProvider().provide_esmini_wrapper()
esmini_wrapper.min_time = 15
esmini_wrapper.max_time = 120.0
esmini_wrapper.grace_time = 1.0
esmini_wrapper.ignored_level = EStoryBoardElementType.ACT
esmini_wrapper.log_to_console = False
esmini_wrapper.log_to_file = False
esmini_wrapper.random_seed = 0

In [ ]:
if run_viewer:
    wrapper = EsminiWrapperProvider().provide_esmini_wrapper()
    wrapper.grace_time = None
    wrapper.max_time = 60.0
    wrapper.ignored_level = EStoryBoardElementType.ACT
    wrapper.view_scenario(scenario_path)

In [ ]:
pps_builder = PPSBuilder()
pps_builder.goal_state_time_step = AbsRel(Interval(-10, 0), False)
pps_builder.goal_state_position_length = 50
pps_builder.goal_state_position_width = 10
pps_builder.goal_state_position_use_ego_rotation = True
pps_builder.goal_state_orientation = None
pps_builder.goal_state_velocity = None
# If you want to also include orientation and velocity in the GoalRegion provide some AbsRel objects
# pps_builder.goal_state_orientation = AbsRel(AngleInterval(-0.8, 0.8), is_absolute=False)
# pps_builder.goal_state_velocity = AbsRel(Interval(-10, 10), is_absolute=False)

In [ ]:

converter = Osc2CrConverter(
    author="ADD AUTHOR HERE",
    affiliation="ADD AFFILIATION HERE",
    source="ADD SOURCE HERE",
    tags={Tag.SIMULATED},
)

converter.sim_wrapper = esmini_wrapper
converter.pps_builder = pps_builder

converter.dt_cr = 0.1
converter.keep_ego_vehicle = True
converter.trim_scenario = False
converter.use_implicit_odr_file = True
converter.analyzers = {
    EAnalyzer.SPOT: SpotAnalyzer(stride=1),
    EAnalyzer.DRIVABILITY: None,  # Will be initialized with default parameters
    EAnalyzer.STL: None,  # Will be initialized with default parameters
}
# If you only want to run with default parameters for analyzers you can also use:
# converter.analyzers = [EAnalyzer.SPOT_ANALYZER, EAnalyzer.DRIVABILITY_CHECKER, EAnalyzer.STL_MONITOR]

converter.dt_sim = 0.01
converter.odr_file_override = None
converter.ego_filter = re.compile(r".*ego.*", re.IGNORECASE)

In [ ]:
result = converter.run_conversion(scenario_path)

In [ ]:
assert isinstance(result, Osc2CrConverterResult)

In [ ]:

count = 0

scenario = result.scenario
pps = result.planning_problem_set
analysis = result.analysis
stats = result.statistics

lim_y = 30
lim_x = lim_y * 3
size = 10

rnd = MPRenderer(figsize=(size, size), plot_limits=[-15, 495, -60, 110])
scenario.draw(rnd, draw_params={
    "trajectory": {
        "draw_continuous": True,
        "line_width": 1,
        "unique_colors": True,
    }
})
pps.draw(rnd)
rnd.render()
plt.savefig("overview.png")
plt.show()

times = [70, 85, 110]

for t in times:
    rnd = MPRenderer(
        figsize=(size, size),
        draw_params={"focus_obstacle_id": scenario.dynamic_obstacles[0].obstacle_id},
        plot_limits=[-lim_x, lim_x, -lim_y, lim_y],
    )
    scenario.draw(rnd, draw_params={
        "trajectory": {
            "draw_continuous": True,
            "line_width": 1,
            "unique_colors": True,
        },
        "time_begin": t
    })
    pps.draw(rnd)
    rnd.render()
    plt.savefig(f"step-{t:2d}.png")
    plt.show()

run_and_break = False
if run_and_break:
    if EAnalyzer.SPOT in analysis:
        res = analysis[EAnalyzer.SPOT][stats.ego_vehicle]
        assert isinstance(res, SpotAnalyzerResult)
        for o_id, o_prediction in res.predictions.items():
            scenario.obstacle_by_id(o_id).prediction = o_prediction
        scenario.obstacle_by_id(49).prediction = None

    rnd = MPRenderer(figsize=(size, size), plot_limits=[-5, 55, -10, 10], )
    scenario.draw(rnd, draw_params={
        "trajectory": {
            "draw_continuous": True,
            "line_width": 1,
            "unique_colors": True,
        }
    })
    pps.draw(rnd)
    rnd.render()
    plt.savefig(f"spot.png")
    plt.show()


